In [4]:
pip install python-docx

Note: you may need to restart the kernel to use updated packages.


In [ ]:
## RUN THE PIPELINE

In [4]:
# run_gui.py - Visually Enhanced with Execution Timer

import tkinter as tk
from tkinter import ttk, scrolledtext, END
import sys
import threading
from io import StringIO
import time # Import the time module

# --- Configuration & Imports ---

# IMPORTANT: Ensure 'pipeline_orchestrator.py' is in the same directory as this file.
try:
    # We import the main pipeline function from the orchestrator
    from pipeline_orchestrator import main_pipeline
except ImportError as e:
    # Use original stderr for robust error reporting during import
    sys.stderr.write(f"Error: Could not import pipeline_orchestrator.py. Ensure the file exists.\nDetails: {e}\n")
    sys.exit(1)

# --- Utility Class for Console Redirection ---

class StreamRedirector:
    """A utility class to redirect stdout/stderr to a Tkinter Text widget."""
    def __init__(self, widget, tag=None):
        self.widget = widget
        self.tag = tag

    def write(self, text):
        # Use master.after to safely update the GUI from a separate thread
        self.widget.after(0, lambda: self._insert_text(text))
        
    def _insert_text(self, text):
        # Check if widget still exists before attempting to write (safety measure)
        if not self.widget.winfo_exists():
            return
            
        self.widget.config(state=tk.NORMAL)
        
        # Apply specific tags based on message content for better color coding
        tag_to_use = self.tag
        if tag_to_use == "error":
            tag_to_use = 'error'
        elif 'PIPELINE EXECUTION COMPLETE' in text or '✅' in text:
            tag_to_use = 'success'
        elif 'Time Taken' in text or 'Summary' in text:
            tag_to_use = 'summary' # New tag for time summary
        elif '[WARNING]' in text or 'Note:' in text:
            tag_to_use = 'warning'
        elif '[INFO]' in text:
            tag_to_use = 'info'
        else:
            tag_to_use = 'default' # Fallback tag for general stdout

        self.widget.insert(tk.END, text, tag_to_use)
        self.widget.yview(tk.END) # Auto-scroll to the end
        self.widget.config(state=tk.DISABLED)
        
    def flush(self):
        pass # Required for file-like objects

# --- Main GUI Class ---

class PipelineGUI:
    """
    A simple Tkinter application to start the agentic pipeline 
    and display its console output in the GUI window.
    """
    def __init__(self, master):
        self.master = master
        master.title("AI-Agent Run Module")
        master.geometry("900x650") 
        
        # Internal variable to track the pipeline start time
        self.pipeline_start_time = None
        
        # Configure the grid layout
        master.columnconfigure(0, weight=1)
        master.rowconfigure(1, weight=1)
        
        # --- 1. Header and Button Frame ---
        self.header_frame = ttk.Frame(master, padding="20 20 20 10")
        self.header_frame.grid(row=0, column=0, sticky="ew")
        self.header_frame.columnconfigure(0, weight=1)
        
        self.label = ttk.Label(self.header_frame, 
                               text="AI-Agent Pipeline",
                               font=("Segoe UI", 24, "bold"),
                               foreground="#333333")
        self.label.pack(pady=(0, 10))
        
        self.run_button = ttk.Button(self.header_frame, 
                                     text="Run Pipeline", 
                                     command=self.start_pipeline_thread,
                                     style="Run.TButton")
        self.run_button.pack(pady=10)
        
        # --- 2. Output Console (ScrolledText) ---
        self.output_frame = ttk.LabelFrame(master, text="Agent Console Output (Live Log)", padding="10")
        self.output_frame.grid(row=1, column=0, sticky="nsew", padx=20, pady=10)
        self.output_frame.columnconfigure(0, weight=1)
        self.output_frame.rowconfigure(0, weight=1)

        self.console_output = scrolledtext.ScrolledText(self.output_frame, 
                                                        wrap=tk.WORD, 
                                                        height=25, 
                                                        state='disabled',
                                                        font=('Consolas', 10),
                                                        bg="darkslategrey", 
                                                        fg="#e0e0e0", 
                                                        insertbackground="grey",
                                                        borderwidth=0,
                                                        relief="flat")
        self.console_output.grid(row=0, column=0, sticky="nsew")

        # Set up color tags for the console output
        self.console_output.tag_config('error', foreground='#ff6961') 
        self.console_output.tag_config('success', foreground='#77dd77') 
        self.console_output.tag_config('warning', foreground='#fdfd96') 
        self.console_output.tag_config('info', foreground='#add8e6') 
        self.console_output.tag_config('summary', foreground='#ffb347', font=('Consolas', 10, 'bold')) # Orange for time
        self.console_output.tag_config('default', foreground='#ffffff') # Default white text
        
        # Redirect stdout and stderr to the console widget
        self.stdout_redirector = StreamRedirector(self.console_output, tag="default")
        self.stderr_redirector = StreamRedirector(self.console_output, tag="error")
        sys.stdout = self.stdout_redirector
        sys.stderr = self.stderr_redirector

    def start_pipeline_thread(self):
        """Disables the button, clears the console, sets the start time, and starts the thread."""
        self.run_button.config(state=tk.DISABLED, text="⏳ Pipeline Running... Do not close.")
        
        self.console_output.config(state=tk.NORMAL)
        self.console_output.delete(1.0, END)
        self.console_output.config(state=tk.DISABLED)
        
        # --- Start Timer ---
        self.pipeline_start_time = time.time()
        print(f"[INFO] Pipeline execution started at {time.strftime('%H:%M:%S', time.localtime(self.pipeline_start_time))}.")
        print("[WARNING] Note: The window may appear unresponsive during model loading/API calls.\n")
        
        # Start the pipeline in a new thread
        self.thread = threading.Thread(target=self.run_pipeline)
        self.thread.daemon = True
        self.thread.start()

    def run_pipeline(self):
        """The core function to execute the orchestrator's pipeline."""
        try:
            main_pipeline()
        except Exception as e:
            print(f"\n❌ CRITICAL ERROR IN ORCHESTRATOR: {e}", file=sys.stderr)
        finally:
            # --- Stop Timer and Calculate Duration ---
            end_time = time.time()
            duration_seconds = end_time - self.pipeline_start_time
            
            # Format duration into H:M:S format
            hours, remainder = divmod(duration_seconds, 3600)
            minutes, seconds = divmod(remainder, 60)
            
            time_str = f"{int(hours)}h {int(minutes)}m {seconds:.2f}s"
            
            # Print summary messages to the console
            print("\n" + "="*70)
            if 'CRITICAL ERROR' not in self.console_output.get(1.0, END):
                 print("✅ PIPELINE EXECUTION COMPLETE. Check your files!")
            
            print(f"\n ⏱️ Total Time Taken: {time_str}")
            print("="*70 + "\n")
            
            # Re-enable the button once the process finishes
            self.master.after(0, self.pipeline_finished)

    def pipeline_finished(self):
        """Called by the main thread when the pipeline thread finishes."""
        self.run_button.config(state=tk.NORMAL, text="✅ Run Complete! Click to Run Again.")
        self.console_output.see(END) # Ensure the final message is visible

if __name__ == "__main__":
    # --- Tkinter Setup ---
    root = tk.Tk()
    
    # Use a modern theme that supports custom styling well
    style = ttk.Style(root)
    try:
        style.theme_use('clam') 
    except tk.TclError:
        style.theme_use('alt') 

    # Custom Style for the Run Button (Run.TButton)
    style.configure("Run.TButton", 
                    font=("Segoe UI", 12, "bold"), 
                    padding=10, 
                    foreground="#ffffff",
                    background="#4CAF50", 
                    borderwidth=0,
                    relief="flat")
    
    try:
        style.map("Run.TButton", 
                 background=[('active', '#66BB6A'), ('disabled', '#A5D6A7')],
                 foreground=[('disabled', '#333333')])
    except tk.TclError:
        pass

    # Configure the LabelFrame border and background
    style.configure("TLabelframe", 
                    background="#f0f0f0", 
                    borderwidth=1, 
                    relief="solid")
    style.configure("TLabelframe.Label", 
                    font=("Segoe UI", 10, "bold"), 
                    foreground="#555555")
    
    root.configure(background="#f0f0f0")

    app = PipelineGUI(root)
    root.mainloop()